In [35]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [36]:
url = 'https://www.flipkart.com/search?q=lenovo%20legion&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off'

In [37]:
def switch_tab(source,tab_index):
    link = source.get('href')
    driver.execute_script(f'window.open("{link}", "_blank");')

    # Wait for the new window to open and then switch to it
    WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > tab_index)
    driver.switch_to.window(driver.window_handles[tab_index])

    # Wait for the new tab to load its content
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'fonts-loaded'))
    )

    # Get the HTML content of the new tab
    new_tab_content = driver.page_source
    soup = BeautifulSoup(new_tab_content, 'html.parser')

    return soup

In [38]:
# Set up the Selenium WebDriver
driver = webdriver.Chrome()
driver.maximize_window()

# Load the page
driver.get(url)

# Wait for the page to fully load
time.sleep(5)

# Get the HTML content of the page
html_content = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Find elements containing product information
products = soup.find_all('a', rel='noopener noreferrer', limit=20)

data = []

for product in products:
    product_details = {}
    name = product.find('div', class_='KzDlHZ')
    name = name.text if name else "No name available"

    # Extract the price of the product
    price = product.find('div', class_='Nx9bqj _4b5DiR')
    price = price.text if price else "No price available"

    # Extract the rating of the product
    rating = product.find('div', class_='_5OesEi')
    rating = rating.text if rating else "No rating available"

    num_ratings_reviews_container = product.find('span', class_='Wphh3N')
    if num_ratings_reviews_container:
        ratings_reviews_text = num_ratings_reviews_container.get_text().strip()
        
        # Split the text to isolate ratings and reviews
        ratings_text, reviews_text = ratings_reviews_text.split('&')
        
        # Extract just the numbers
        num_ratings = ''.join(filter(str.isdigit, ratings_text))
        num_reviews = ''.join(filter(str.isdigit, reviews_text))
    else:
        num_ratings = "No ratings"
        num_reviews = "No reviews"

    # print(f"Name: {name}")
    # print(f"Price: {price}")
    # print(f"Rating: {rating}")
    # print(f"Number of Ratings: {num_ratings}")
    # print(f"Number of Reviews: {num_reviews}")
    # print('-' * 50)
    
    product_details['Product Name'] = name
    product_details['Price'] = price
    product_details['Rating'] = rating
    product_details['Number of Ratings'] = num_ratings
    product_details['Number of Reviews'] = num_reviews
    if num_reviews != "No reviews" and num_reviews != "0":
        soup = switch_tab(product,1)
        # Find the element with the reviews or link to reviews
        div_element = soup.find('div', class_='RcXBOT')

        if div_element:
            # Find all <a> tags within the div
            all_a_tags = div_element.find_all('a')
            # Check if there are any <a> tags
            if len(all_a_tags) > 10:
                # Get the last <a> tag
                last_a_tag = all_a_tags[-1]

                soup = switch_tab(last_a_tag,2)

                reviews_container = soup.find_all('div',class_='col EPCmJX')
                review_summary_list = []
                review_details_list = []
                for review in reviews_container:
                    review_summary = review.find('p',class_='z9E0IG').text if review.find('p',class_='z9E0IGT') else "No summary"
                    review_details = review.find('div',class_='ZmyHeo').text if review.find('div',class_='ZmyHeo') else "No details"
                    review_summary_list.append(review_summary)
                    review_details_list.append(review_details)
                    # print(f"{reviews_container.index(review)}. review_summary: {review_summary}")
                    # print(f"{reviews_container.index(review)}. review_details: {review_details}")
                product_details['Review Summary'] = review_summary_list
                product_details['Review Details'] = review_details_list
                driver.close()
            else:
                review_summary_list = []
                review_details_list = []
                reviews_summary_container = soup.find_all('p',class_='z9E0IG')
                for reviews_summary in reviews_summary_container:
                    reviews_summary = reviews_summary.text
                    review_summary_list.append(reviews_summary)
                #     # print(f"{reviews_summary_container.index(reviews_summary)}. review_summary: {review_summary}")
                reviews_details_container = soup.find_all('div',class_='ZmyHeo')
                for review_details in reviews_details_container:
                    # print(reviews_details_container.index(review_details))
                    review_details = review_details.text
                    review_details_list.append(review_details)
                    # print(f". review_details: {review_details}")
                product_details['Review Summary'] = review_summary_list
                product_details['Review Details'] = review_details_list

            # print('-' *50)
            
            driver.switch_to.window(driver.window_handles[1])
            driver.close()
        driver.switch_to.window(driver.window_handles[0])
    data.append(product_details)

# Close the browser
driver.quit()

In [39]:
data

[{'Product Name': 'Lenovo Legion 5 AMD AMD Ryzen 7 Octa Core 5800H - (16 GB/512 GB SSD/Windows 11 Home/6 GB Graphics/NVID...',
  'Price': '₹99,990',
  'Rating': '4.392 Ratings\xa0&\xa018 Reviews',
  'Number of Ratings': '92',
  'Number of Reviews': '18',
  'Review Summary': ['Good quality product',
   'Highly recommended',
   'Worth every penny',
   'Classy product',
   'Very poor',
   'Good choice',
   'Wonderful',
   'Must buy!',
   'Delightful',
   'Really Nice'],
  'Review Details': ["Very powerful gaming machine, but I highly recommend u to use cooling pads, as temperature rise above 100degrees .I was playing uncharted 4 in highest possible settings, I could easily get 60+ fps, but the CPU temp reached 100degrees.Another thing I noticed is the hinges are poor quality,i don't get the robust feel in it, it's been just 1 month and the hinges feel like I have opened and closed it for years and made it looseREAD MORE",
   'its just a beastREAD MORE',
   "It's an amazing product. I love

In [40]:
df = pd.DataFrame(data)
df

,Product Name,Price,Rating,Number of Ratings,Number of Reviews,Review Summary,Review Details
0,Lenovo Legion 5 AMD AMD Ryzen 7 Octa Core 5800...,"₹99,990",4.392 Ratings & 18 Reviews,92,18,"[Good quality product, Highly recommended, Wor...","[Very powerful gaming machine, but I highly re..."
1,Lenovo Legion Intel Core i9 14th Gen 14900HX -...,"₹3,74,990",No rating available,No ratings,No reviews,NaN,NaN
2,Lenovo Lenovo Legion Slim 5 AI Powered AMD Ryz...,"₹1,14,190",4.85 Ratings & 0 Reviews,5,0,NaN,NaN
3,Lenovo Lenovo Legion Slim 5 AI Powered AMD Ryz...,"₹1,32,190",3.33 Ratings & 1 Reviews,3,1,[Terrific],[Excellent in all respects.READ MORE]
4,Lenovo Lenovo Legion Slim 5 AI Powered AMD Ryz...,"₹1,28,990",44 Ratings & 0 Reviews,4,0,NaN,NaN
5,Lenovo Legion Intel Core i9 13th Gen 13980HX -...,"₹4,20,990",No rating available,No ratings,No reviews,NaN,NaN
6,Lenovo Lenovo Legion Slim 5 AI Powered AMD Ryz...,"₹1,30,990",44 Ratings & 0 Reviews,4,0,NaN,NaN
7,Lenovo Legion 5 AMD AMD Ryzen 7 Octa Core 5800...,"₹1,15,990",4.465 Ratings & 7 Reviews,65,7,"[Simply awesome, Worth every penny, Terrific, ...",[Product is GREAT! Few thing buyers should not...
8,Lenovo Legion 5 Pro AMD Ryzen 7 Octa Core 5800...,"₹89,990",4.481 Ratings & 7 Reviews,81,7,"[Absolute rubbish!, Mind-blowing purchase, Won...",[1 star rating is for Flipkart's smart upgrade...
9,Lenovo Legion Intel Core i7 14th Gen 14650HX -...,"₹2,16,190",No rating available,No ratings,No reviews,NaN,NaN


In [45]:
df.to_csv('D:\\projects\\webscrapings.csv', index=False)
